# Entry ? - N-grams

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px

import string
import re
import nltk
import emoji
import itertools

In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_predict
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Functions

## Filter data, create features

In [3]:
def count_ratios(df, text_col):
    df['char_count'] = df[text_col].str.len()
    df['word_count'] = df[text_col].str.count('\\w+')
    df['ltr_count'] = df[text_col].str.count('[A-Za-z]')
    df['ltr_ratio'] = (df['ltr_count'] / df['char_count']).fillna(0)
    df['spec_char_count'] = df[text_col].apply(lambda x: sum(map(x.count, string.punctuation)))
    df['spec_char_ratio'] = (df['spec_char_count'] / df['char_count']).fillna(0)
    df['num_count'] = df[text_col].str.count('[0-9]+')
    df['num_ratio'] = (df['num_count'] / df['char_count']).fillna(0)
    df['vowel_count'] = df[text_col].str.count('[aeiouyAEIOUY]')
    df['vowel_ratio'] = (df['vowel_count'] / df['char_count']).fillna(0)
    df['caps_count'] = df[text_col].str.count('[A-Z]')
    df['caps_ratio'] = (df['caps_count'] / df['char_count']).fillna(0)
    df['newline_tab_count'] = df[text_col].str.count(r'[\t\r\n]')
    df['newline_tab_ratio'] = (df['newline_tab_count'] / df['char_count']).fillna(0)
    df['qwerty_count'] = df[text_col].str.count('[asdfghjkl]')
    df['qwerty_ratio'] = (df['qwerty_count'] / df['char_count']).fillna(0)
    return df
    
def punct_tokens(df, text_col):
    newline_list = '\t\r\n'
    remove_newline = str.maketrans(' ', ' ', newline_list)
    emoji_string = ''.join(emoji.UNICODE_EMOJI.keys())
    punct_list = string.punctuation + '-‘_”' + emoji_string
    nopunct = str.maketrans('', '', punct_list)
    df['no_punct_tokens'] = df[text_col].fillna("").str.lower().str.translate(remove_newline).str.translate(nopunct).str.split()
    df['distinct_word_count'] = df['no_punct_tokens'].apply(lambda x: len(set(x)))
    df['max_word_len'] = df['no_punct_tokens'].apply(lambda x: max([len(word) for word in x], default=0))
    df['min_word_len'] = df['no_punct_tokens'].apply(lambda x: min([len(word) for word in x], default=0))
    df['word_len_range'] = df['max_word_len'] - df['min_word_len']
    df['word_diversity'] = (df['distinct_word_count'] / df['word_count']).fillna(0)
    df['avg_word_len'] = (df['char_count'] / df['word_count']).fillna(0)
    df['repeat_ltrs'] = df['no_punct_tokens'].apply(lambda x: [word for word in x if re.search(r'([a-zA-Z])\1{2,}', word.lower())])
    df['repeat_ltr_count'] = df[text_col].str.count(r'([a-zA-Z])\1{2,}')
    df['repeat_ltr_ratio'] = (df['repeat_ltr_count'] / df['word_count']).fillna(0)
    return df

def tribi_grams(df):
    stop = nltk.corpus.stopwords.words('english')
    df['unigrams'] = df['no_punct_tokens'].apply(lambda x: [item for item in x if item not in stop])
    df['bigrams'] = df['unigrams'].apply(lambda x:(list(nltk.bigrams(x))))
    df['trigrams'] = df['unigrams'].apply(lambda x:(list(nltk.trigrams(x))))
#     df['spam_unigram'] = df.apply(lambda x: [[x['spam'],word] for word in x['no_punct_tokens'], axis=1])
#     df['spam_bigram'] = df.apply(lambda x: [[x['spam'],word] for word in x['bigrams'], axis=1])
#     df['spam_trigram'] = df.apply(lambda x: [[x['spam'],word] for word in x['trigrams'], axis=1])
    return df

def clean_features(df, text_col):
    df = count_ratios(df, text_col)
    df = punct_tokens(df, text_col)
    df.loc[df['avg_word_len'] == np.inf, 'avg_word_len'] = 0
    df = tribi_grams(df)
    return df

In [4]:
sms_df = pd.read_csv('data/smsspamcollection/SMSSpamCollection', sep='\t', names =['spam', 'text'])
sms_df = clean_features(sms_df, 'text')

In [5]:
sms_df['target'] = sms_df['spam'].map({'ham': 0, 'spam': 1})

In [6]:
sms_df.head(3)

,spam,text,char_count,word_count,ltr_count,ltr_ratio,spec_char_count,spec_char_ratio,num_count,num_ratio,...,word_len_range,word_diversity,avg_word_len,repeat_ltrs,repeat_ltr_count,repeat_ltr_ratio,unigrams,bigrams,trigrams,target
0,ham,"Go until jurong point, crazy.. Available only ...",111,20,83,0.747748,9,0.081081,0,0.00000,...,8,1.000000,5.550000,[],0,0.0,"[go, jurong, point, crazy, available, bugis, n...","[(go, jurong), (jurong, point), (point, crazy)...","[(go, jurong, point), (jurong, point, crazy), ...",0
1,ham,Ok lar... Joking wif u oni...,29,6,18,0.620690,6,0.206897,0,0.00000,...,5,1.000000,4.833333,[],0,0.0,"[ok, lar, joking, wif, u, oni]","[(ok, lar), (lar, joking), (joking, wif), (wif...","[(ok, lar, joking), (lar, joking, wif), (jokin...",0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,33,97,0.625806,6,0.038710,6,0.03871,...,10,0.636364,4.696970,[],0,0.0,"[free, entry, wkly, comp, win, fa, cup, final,...","[(free, entry), (entry, wkly), (wkly, comp), (...","[(free, entry, wkly), (entry, wkly, comp), (wk...",1


In [7]:
sms_df[['target', 'text', 'unigrams', 'bigrams', 'trigrams']].head(3)

,target,text,unigrams,bigrams,trigrams
0,0,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n...","[(go, jurong), (jurong, point), (point, crazy)...","[(go, jurong, point), (jurong, point, crazy), ..."
1,0,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[(ok, lar), (lar, joking), (joking, wif), (wif...","[(ok, lar, joking), (lar, joking, wif), (jokin..."
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, wkly, comp, win, fa, cup, final,...","[(free, entry), (entry, wkly), (wkly, comp), (...","[(free, entry, wkly), (entry, wkly, comp), (wk..."


In [8]:
sms_df.columns

Index(['spam', 'text', 'char_count', 'word_count', 'ltr_count', 'ltr_ratio',
       'spec_char_count', 'spec_char_ratio', 'num_count', 'num_ratio',
       'vowel_count', 'vowel_ratio', 'caps_count', 'caps_ratio',
       'newline_tab_count', 'newline_tab_ratio', 'qwerty_count',
       'qwerty_ratio', 'no_punct_tokens', 'distinct_word_count',
       'max_word_len', 'min_word_len', 'word_len_range', 'word_diversity',
       'avg_word_len', 'repeat_ltrs', 'repeat_ltr_count', 'repeat_ltr_ratio',
       'unigrams', 'bigrams', 'trigrams', 'target'],
      dtype='object')

In [7]:
X = sms_df.drop(['spam', 'target'], axis=1)
y = sms_df['target']
X_ngrams, X, y_ngrams, y = train_test_split(X, y, train_size=0.2, random_state=12)

In [8]:
ngram_df = pd.concat([X_ngrams, y_ngrams], axis=1)[['text', 'unigrams', 'bigrams', 'trigrams', 'target']]

In [9]:
ngram_df.head()

,text,unigrams,bigrams,trigrams,target
3442,Get a FREE mobile video player FREE movie. To ...,"[get, free, mobile, video, player, free, movie...","[(get, free), (free, mobile), (mobile, video),...","[(get, free, mobile), (free, mobile, video), (...",1
4172,Pls what's the full name of joke's school cos ...,"[pls, whats, full, name, jokes, school, cos, f...","[(pls, whats), (whats, full), (full, name), (n...","[(pls, whats, full), (whats, full, name), (ful...",0
1943,K...k:)why cant you come here and search job:),"[kkwhy, cant, come, search, job]","[(kkwhy, cant), (cant, come), (come, search), ...","[(kkwhy, cant, come), (cant, come, search), (c...",0
2089,Well done ENGLAND! Get the official poly ringt...,"[well, done, england, get, official, poly, rin...","[(well, done), (done, england), (england, get)...","[(well, done, england), (done, england, get), ...",1
2359,I'll talk to the others and probably just come...,"[ill, talk, others, probably, come, early, tom...","[(ill, talk), (talk, others), (others, probabl...","[(ill, talk, others), (talk, others, probably)...",0


In [35]:
def create_ngram_dist(df, ngram_col, target_col):
    total_ngram_list = list(itertools.chain(*df[ngram_col].ravel()))
    total_freq = nltk.FreqDist(total_ngram_list)
    total_freq_df = pd.DataFrame.from_dict(total_freq, orient='index', columns=['total_ct'])
    tar_gram_list = list(itertools.chain(*df[ngram_df[target_col] == 1][ngram_col].ravel()))
    tar_freq = nltk.FreqDist(tar_gram_list)
    tar_freq_df = pd.DataFrame.from_dict(tar_freq, orient='index', columns=['tar_ct'])
    freq_df = pd.merge(total_freq_df, tar_freq_df, how='left', left_index=True, right_index=True).fillna(0)
    freq_df['tar_rate'] = freq_df['tar_ct'] / freq_df['total_ct']
    freq_df['log2_total_ct'] = np.log2(freq_df['total_ct'])
    freq_df['log_total_ct'] = np.log(freq_df['total_ct'])
    freq_df['log10_total_ct'] = np.log10(freq_df['total_ct'])
    freq_df['log2_tar_score'] = freq_df['log2_total_ct'] * freq_df['tar_rate']
    freq_df['log_tar_score'] = freq_df['log_total_ct'] * freq_df['tar_rate']
    freq_df['log10_tar_score'] = freq_df['log10_total_ct'] * freq_df['tar_rate']
    return freq_df

In [36]:
unigram_dist_df = create_ngram_dist(ngram_df, 'unigrams', 'target')
bigram_dist_df = create_ngram_dist(ngram_df, 'bigrams', 'target')
trigram_dist_df = create_ngram_dist(ngram_df, 'trigrams', 'target')

In [37]:
unigram_dist_df.sort_values('tar_ct')

,total_ct,tar_ct,tar_rate,log2_total_ct,log_total_ct,log10_total_ct,log2_tar_score,log_tar_score,log10_tar_score
nap,2,0.0,0.000000,1.000000,0.693147,0.301030,0.000000,0.000000,0.000000
hand,5,0.0,0.000000,2.321928,1.609438,0.698970,0.000000,0.000000,0.000000
wit,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
clear,2,0.0,0.000000,1.000000,0.693147,0.301030,0.000000,0.000000,0.000000
copies,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
ur,85,36.0,0.423529,6.409391,4.442651,1.929419,2.714566,1.881593,0.817166
txt,40,37.0,0.925000,5.321928,3.688879,1.602060,4.922783,3.412213,1.481905
free,64,53.0,0.828125,6.000000,4.158883,1.806180,4.968750,3.444075,1.495743
£,54,54.0,1.000000,5.754888,3.988984,1.732394,5.754888,3.988984,1.732394


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=12)

In [24]:
X_train.shape

(3566, 30)

In [26]:
X_train.head(3)

,text,char_count,word_count,ltr_count,ltr_ratio,spec_char_count,spec_char_ratio,num_count,num_ratio,vowel_count,...,min_word_len,word_len_range,word_diversity,avg_word_len,repeat_ltrs,repeat_ltr_count,repeat_ltr_ratio,unigrams,bigrams,trigrams
1938,The fact that you're cleaning shows you know w...,133,28,99,0.744361,9,0.067669,0,0.0,42,...,1,9,0.785714,4.750000,[],0,0.0,"[fact, youre, cleaning, shows, know, im, upset...","[(fact, youre), (youre, cleaning), (cleaning, ...","[(fact, youre, cleaning), (youre, cleaning, sh..."
495,Are you free now?can i call now?,32,8,24,0.750000,2,0.062500,0,0.0,12,...,1,5,0.875000,4.000000,[],0,0.0,"[free, nowcan, call]","[(free, nowcan), (nowcan, call)]","[(free, nowcan, call)]"
3798,For The First Time In The History 'Need' 'Comf...,180,33,115,0.638889,36,0.200000,0,0.0,39,...,2,6,0.636364,5.454545,[],0,0.0,"[first, time, history, need, comfort, luxury, ...","[(first, time), (time, history), (history, nee...","[(first, time, history), (time, history, need)..."


In [27]:
X_train[['unigrams']].map()

,unigrams
1938,"[fact, youre, cleaning, shows, know, im, upset..."
495,"[free, nowcan, call]"
3798,"[first, time, history, need, comfort, luxury, ..."
1876,"[hmv, bonus, special, pounds, genuine, hmv, vo..."
2580,"[dunno, lets, go, learn, pilates]"
...,...
2597,"[im, gonna, able, late, notice, ill, home, wee..."
4568,"[together, thinkin]"
2996,"[mm, entirely, sure, understood, text, hey, ho..."
5161,"[lol, need, cash, nitros, hurry, come, crash]"


In [85]:
X_train['unigrams'].map(lambda r:
                        list(itertools.chain(*[unigram_dist_df.loc[unigram_dist_df.index == x, 'total_ct'] for x in r if x in unigram_dist_df.index])))

1938                [1, 14, 8, 48, 105, 44, 42]
495                                   [64, 103]
3798            [13, 39, 42, 5, 41, 41, 41, 41]
1876    [2, 2, 13, 4, 2, 4, 4, 5, 1, 10, 34, 2]
2580                           [8, 4, 49, 1, 2]
                         ...                   
2597       [105, 18, 5, 10, 2, 44, 31, 6, 6, 4]
4568                                     [3, 2]
2996                      [3, 21, 38, 24, 1, 3]
5161                        [11, 42, 13, 3, 49]
3387               [1, 5, 4, 34, 5, 27, 10, 16]
Name: unigrams, Length: 3566, dtype: object

In [209]:
temp = pd.DataFrame(X_train['unigrams'].values.tolist())

In [210]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,fact,youre,cleaning,shows,know,im,upset,priority,constantly,want,...,None,None,None,None,None,None,None,None,None,None
1,free,nowcan,call,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,first,time,history,need,comfort,luxury,sold,price,india,onionrs,...,None,None,None,None,None,None,None,None,None,None
3,hmv,bonus,special,pounds,genuine,hmv,vouchers,answer,easy,questions,...,None,None,None,None,None,None,None,None,None,None
4,dunno,lets,go,learn,pilates,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [215]:
for col in temp.columns:
    temp[col] = temp[col].map(unigram_dist_df['total_ct'])
combined_df = pd.DataFrame()
combined_df['uni_total_ct'] = temp2.values.tolist()
combined_df['uni_total_ct'] = combined_df['uni_total_ct'].apply(lambda r: [x for x in r if ~np.isnan(x)])

In [216]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
combined_df.head()

,uni_total_ct
0,"[1.0, 14.0, 8.0, 48.0, 105.0, 44.0, 42.0]"
1,"[64.0, 103.0]"
2,"[13.0, 39.0, 42.0, 5.0, 41.0, 41.0, 41.0, 41.0]"
3,"[2.0, 2.0, 13.0, 4.0, 2.0, 4.0, 4.0, 5.0, 1.0,..."
4,"[8.0, 4.0, 49.0, 1.0, 2.0]"


In [202]:
# for x in temp3.loc[0, 'uni_total_ct']:
#     if ~np.isnan(x):
#         print(x)

# [x for x in temp3.loc[0, 'uni_total_ct'] if ~np.isnan(x)]

In [208]:
combined_df.head()

,uni_total_ct
0,"[1.0, 14.0, 8.0, 48.0, 105.0, 44.0, 42.0]"
1,"[64.0, 103.0]"
2,"[13.0, 39.0, 42.0, 5.0, 41.0, 41.0, 41.0, 41.0]"
3,"[2.0, 2.0, 13.0, 4.0, 2.0, 4.0, 4.0, 5.0, 1.0,..."
4,"[8.0, 4.0, 49.0, 1.0, 2.0]"


In [219]:
%%timeit
temp = pd.DataFrame(X_train['unigrams'].values.tolist())
for col in temp.columns:
    temp[col] = temp[col].map(unigram_dist_df['total_ct'])
combined_df = pd.DataFrame()
combined_df['uni_total_ct'] = temp2.values.tolist()
combined_df['uni_total_ct'] = combined_df['uni_total_ct'].apply(lambda r: [x for x in r if ~np.isnan(x)])

559 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [220]:
%%timeit
combined2 = pd.DataFrame()
combined2['uni_total_ct'] = X_train['unigrams'].map(lambda r:
                                                    list(itertools.chain(*[unigram_dist_df.loc[
                                                        unigram_dist_df.index == x, 'total_ct'
                                                    ] for x in r if x in unigram_dist_df.index])))

7.6 s ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [221]:
unigram_dist_df.columns

Index(['total_ct', 'tar_ct', 'tar_rate', 'log2_total_ct', 'log_total_ct',
       'log10_total_ct', 'log2_tar_score', 'log_tar_score', 'log10_tar_score'],
      dtype='object')

In [222]:
ngram_cols = ['unigrams', 'bigrams', 'trigrams']
val_map_cols = unigram_dist_df.columns

In [244]:
def create_val_map_df(df, dist_df):
    combined_df = pd.DataFrame()
    val_map_cols = dist_df.columns
    word_df = pd.DataFrame(df['unigrams'].values.tolist())
    for val_col in val_map_cols:
        temp=pd.DataFrame()
        for col in word_df.columns:            
            temp[col] = word_df[col].map(dist_df[val_col])
        col_name = 'uni_' + val_col
        combined_df[col_name] = temp.values.tolist()
        combined_df[col_name] = combined_df[col_name].apply(lambda r: [x for x in r if ~np.isnan(x)])
    return combined_df

In [318]:
unigram_vals_df = create_val_map_df(X_train, unigram_dist_df)

In [248]:
unigram_vals_df.head()

,uni_total_ct,uni_tar_ct,uni_tar_rate,uni_log2_total_ct,uni_log_total_ct,uni_log10_total_ct,uni_log2_tar_score,uni_log_tar_score,uni_log10_tar_score
0,"[1.0, 14.0, 8.0, 48.0, 105.0, 44.0, 42.0]","[0.0, 0.0, 8.0, 6.0, 3.0, 8.0, 2.0]","[0.0, 0.0, 1.0, 0.125, 0.02857142857142857, 0....","[0.0, 3.807354922057604, 3.0, 5.58496250072115...","[0.0, 2.6390573296152584, 2.0794415416798357, ...","[0.0, 1.146128035678238, 0.9030899869919435, 1...","[0.0, 0.0, 3.0, 0.6981203125901445, 0.19183558...","[0.0, 0.0, 2.0794415416798357, 0.4839001263634...","[0.0, 0.0, 0.9030899869919435, 0.2101551546719..."
1,"[64.0, 103.0]","[53.0, 59.0]","[0.828125, 0.5728155339805825]","[6.0, 6.6865005271832185]","[4.1588830833596715, 4.634728988229636]","[1.806179973983887, 2.012837224705172]","[4.96875, 3.8301313699399016]","[3.4440750534072277, 2.654844760248044]","[1.4957427909554064, 1.1529844296854868]"
2,"[13.0, 39.0, 42.0, 5.0, 41.0, 41.0, 41.0, 41.0]","[1.0, 3.0, 2.0, 5.0, 0.0, 0.0, 0.0, 0.0]","[0.07692307692307693, 0.07692307692307693, 0.0...","[3.700439718141092, 5.285402218862249, 5.39231...","[2.5649493574615367, 3.6635616461296463, 3.737...","[1.1139433523068367, 1.591064607026499, 1.6232...","[0.28464920908777636, 0.4065694014509422, 0.25...","[0.19730379672781054, 0.2818124343176651, 0.17...","[0.08568795017744898, 0.12238958515588455, 0.0..."
3,"[2.0, 2.0, 13.0, 4.0, 2.0, 4.0, 4.0, 5.0, 1.0,...","[2.0, 2.0, 3.0, 4.0, 2.0, 4.0, 2.0, 1.0, 0.0, ...","[1.0, 1.0, 0.23076923076923078, 1.0, 1.0, 1.0,...","[1.0, 1.0, 3.700439718141092, 2.0, 1.0, 2.0, 2...","[0.6931471805599453, 0.6931471805599453, 2.564...","[0.3010299956639812, 0.3010299956639812, 1.113...","[1.0, 1.0, 0.8539476272633291, 2.0, 1.0, 2.0, ...","[0.6931471805599453, 0.6931471805599453, 0.591...","[0.3010299956639812, 0.3010299956639812, 0.257..."
4,"[8.0, 4.0, 49.0, 1.0, 2.0]","[0.0, 1.0, 5.0, 0.0, 0.0]","[0.0, 0.25, 0.10204081632653061, 0.0, 0.0]","[3.0, 2.0, 5.614709844115208, 0.0, 1.0]","[2.0794415416798357, 1.3862943611198906, 3.891...","[0.9030899869919435, 0.6020599913279624, 1.690...","[0.0, 0.5, 0.5729295759301233, 0.0, 0.0]","[0.0, 0.34657359027997264, 0.39712452021537004...","[0.0, 0.1505149978319906, 0.1724689877580116, ..."


In [327]:
unigram_vals_df.loc[135, :]

uni_total_ct           NaN
uni_tar_ct              []
uni_tar_rate            []
uni_log2_total_ct       []
uni_log_total_ct        []
uni_log10_total_ct      []
uni_log2_tar_score      []
uni_log_tar_score       []
uni_log10_tar_score     []
Name: 135, dtype: object

In [325]:
unigram_vals_df.loc[unigram_vals_df['uni_total_ct'].str.len() == 0, 'uni_total_ct':'uni_log10_tar_score'] = np.nan

In [329]:
np.nanmax(np.nan)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning:

All-NaN axis encountered



nan

In [330]:
unigram_vals_df['uni_total_ct'].apply(lambda x: np.nanmax(x))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning:

All-NaN axis encountered



0       105.0
1       103.0
2        42.0
3        34.0
4        49.0
        ...  
3561    105.0
3562      3.0
3563     38.0
3564     49.0
3565     34.0
Name: uni_total_ct, Length: 3566, dtype: float64

In [251]:
max(unigram_vals_df.loc[0, 'uni_total_ct'])

105.0

In [282]:
unigram_vals_df.shape[0]

3566

In [283]:
for row in range(unigram_vals_df.shape[0]):
    print(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [280]:
unigram_vals_df.loc[0, 'uni_total_ct']

[1.0, 14.0, 8.0, 48.0, 105.0, 44.0, 42.0]

In [293]:
X_train.head()

,text,char_count,word_count,ltr_count,ltr_ratio,spec_char_count,spec_char_ratio,num_count,num_ratio,vowel_count,...,min_word_len,word_len_range,word_diversity,avg_word_len,repeat_ltrs,repeat_ltr_count,repeat_ltr_ratio,unigrams,bigrams,trigrams
1938,The fact that you're cleaning shows you know w...,133,28,99,0.744361,9,0.067669,0,0.000000,42,...,1,9,0.785714,4.750000,[],0,0.000000,"[fact, youre, cleaning, shows, know, im, upset...","[(fact, youre), (youre, cleaning), (cleaning, ...","[(fact, youre, cleaning), (youre, cleaning, sh..."
495,Are you free now?can i call now?,32,8,24,0.750000,2,0.062500,0,0.000000,12,...,1,5,0.875000,4.000000,[],0,0.000000,"[free, nowcan, call]","[(free, nowcan), (nowcan, call)]","[(free, nowcan, call)]"
3798,For The First Time In The History 'Need' 'Comf...,180,33,115,0.638889,36,0.200000,0,0.000000,39,...,2,6,0.636364,5.454545,[],0,0.000000,"[first, time, history, need, comfort, luxury, ...","[(first, time), (time, history), (history, nee...","[(first, time, history), (time, history, need)..."
1876,HMV BONUS SPECIAL 500 pounds of genuine HMV vo...,155,29,112,0.722581,7,0.045161,4,0.025806,42,...,2,19,0.655172,5.344828,[infowwwpercentrealcom],1,0.034483,"[hmv, bonus, special, pounds, genuine, hmv, vo...","[(hmv, bonus), (bonus, special), (special, pou...","[(hmv, bonus, special), (bonus, special, pound..."
2580,I dunno until when... Lets go learn pilates...,46,8,33,0.717391,6,0.130435,0,0.000000,13,...,1,6,1.000000,5.750000,[],0,0.000000,"[dunno, lets, go, learn, pilates]","[(dunno, lets), (lets, go), (go, learn), (lear...","[(dunno, lets, go), (lets, go, learn), (go, le..."


In [294]:
X_train.iloc[135, :]

text                      G.W.R
char_count                    5
word_count                    3
ltr_count                     3
ltr_ratio                   0.6
spec_char_count               2
spec_char_ratio             0.4
num_count                     0
num_ratio                     0
vowel_count                   0
vowel_ratio                   0
caps_count                    3
caps_ratio                  0.6
newline_tab_count             0
newline_tab_ratio             0
qwerty_count                  0
qwerty_ratio                  0
no_punct_tokens           [gwr]
distinct_word_count           1
max_word_len                  3
min_word_len                  3
word_len_range                0
word_diversity         0.333333
avg_word_len            1.66667
repeat_ltrs                  []
repeat_ltr_count              0
repeat_ltr_ratio              0
unigrams                  [gwr]
bigrams                      []
trigrams                     []
Name: 4293, dtype: object

In [291]:
unigram_vals_df.loc[135, :]

uni_total_ct           []
uni_tar_ct             []
uni_tar_rate           []
uni_log2_total_ct      []
uni_log_total_ct       []
uni_log10_total_ct     []
uni_log2_tar_score     []
uni_log_tar_score      []
uni_log10_tar_score    []
Name: 135, dtype: object

In [299]:
for i, row in enumerate(range(unigram_vals_df.shape[0])):
    if unigram_vals_df.loc[row, 'uni_total_ct']:
        print(i, max(unigram_vals_df.loc[row, 'uni_total_ct']))

0 105.0
1 103.0
2 42.0
3 34.0
4 49.0
5 2.0
6 31.0
7 105.0
8 103.0
9 85.0
10 57.0
11 9.0
12 24.0
13 213.0
14 37.0
15 49.0
16 23.0
17 5.0
18 213.0
19 44.0
20 31.0
21 213.0
22 2.0
23 213.0
24 213.0
25 54.0
26 39.0
27 41.0
28 44.0
29 12.0
30 213.0
31 19.0
32 19.0
33 103.0
34 105.0
35 36.0
36 64.0
37 69.0
38 14.0
39 69.0
40 213.0
41 49.0
42 103.0
43 57.0
44 48.0
45 103.0
46 105.0
47 103.0
48 103.0
49 41.0
50 49.0
51 213.0
52 103.0
53 69.0
54 3.0
55 64.0
56 213.0
57 34.0
58 213.0
59 6.0
60 48.0
61 9.0
62 32.0
63 2.0
64 105.0
65 24.0
66 213.0
67 69.0
68 105.0
69 213.0
70 22.0
71 41.0
72 13.0
73 69.0
74 105.0
75 22.0
76 12.0
77 21.0
78 54.0
79 105.0
80 36.0
81 64.0
82 64.0
83 213.0
84 7.0
85 65.0
86 64.0
87 11.0
88 69.0
89 69.0
90 213.0
91 103.0
92 103.0
93 4.0
94 28.0
95 65.0
96 213.0
97 213.0
98 105.0
99 213.0
100 213.0
101 49.0
102 40.0
103 31.0
104 44.0
105 213.0
106 213.0
107 41.0
108 6.0
109 103.0
110 105.0
111 34.0
112 44.0
113 213.0
114 213.0
115 85.0
116 65.0
117 62.0
118 44.0
119 213

In [275]:
for col in unigram_vals_df.columns:
    print(unigram_vals_df[col])

0               [1.0, 14.0, 8.0, 48.0, 105.0, 44.0, 42.0]
1                                           [64.0, 103.0]
2         [13.0, 39.0, 42.0, 5.0, 41.0, 41.0, 41.0, 41.0]
3       [2.0, 2.0, 13.0, 4.0, 2.0, 4.0, 4.0, 5.0, 1.0,...
4                              [8.0, 4.0, 49.0, 1.0, 2.0]
                              ...                        
3561    [105.0, 18.0, 5.0, 10.0, 2.0, 44.0, 31.0, 6.0,...
3562                                           [3.0, 2.0]
3563                    [3.0, 21.0, 38.0, 24.0, 1.0, 3.0]
3564                        [11.0, 42.0, 13.0, 3.0, 49.0]
3565         [1.0, 5.0, 4.0, 34.0, 5.0, 27.0, 10.0, 16.0]
Name: uni_total_ct, Length: 3566, dtype: object
0                     [0.0, 0.0, 8.0, 6.0, 3.0, 8.0, 2.0]
1                                            [53.0, 59.0]
2                [1.0, 3.0, 2.0, 5.0, 0.0, 0.0, 0.0, 0.0]
3       [2.0, 2.0, 3.0, 4.0, 2.0, 4.0, 2.0, 1.0, 0.0, ...
4                               [0.0, 1.0, 5.0, 0.0, 0.0]
                        

In [20]:
for col in unigram_vals_df.columns:
    unigram_vals_df.loc[unigram_vals_df[col].str.len() == 0, col] = np.nan

In [21]:
unigram_vals_df.loc[135, :]

uni_total_ct           NaN
uni_tar_ct             NaN
uni_tar_rate           NaN
uni_log2_total_ct      NaN
uni_log_total_ct       NaN
uni_log10_total_ct     NaN
uni_log2_tar_score     NaN
uni_log_tar_score      NaN
uni_log10_tar_score    NaN
Name: 135, dtype: object

In [227]:
combined_df=pd.DataFrame()
val_map_cols = unigram_dist_df.columns

In [241]:
word_df = pd.DataFrame(X_train['unigrams'].values.tolist())
for val_col in val_map_cols:
    print(val_col)
    temp = pd.DataFrame()
    for col in word_df.columns:
#         temp[col] = word_df[col].map(unigram_dist_df[col])
#     print(temp.head())
        print(word_df[col].head(), unigram_dist_df[val_col].head())

total_ct
0     fact
1     free
2    first
3      hmv
4    dunno
Name: 0, dtype: object get       69
free      64
mobile    20
video      9
player     3
Name: total_ct, dtype: int64
0     youre
1    nowcan
2      time
3     bonus
4      lets
Name: 1, dtype: object get       69
free      64
mobile    20
video      9
player     3
Name: total_ct, dtype: int64
0    cleaning
1        call
2     history
3     special
4          go
Name: 2, dtype: object get       69
free      64
mobile    20
video      9
player     3
Name: total_ct, dtype: int64
0     shows
1      None
2      need
3    pounds
4     learn
Name: 3, dtype: object get       69
free      64
mobile    20
video      9
player     3
Name: total_ct, dtype: int64
0       know
1       None
2    comfort
3    genuine
4    pilates
Name: 4, dtype: object get       69
free      64
mobile    20
video      9
player     3
Name: total_ct, dtype: int64
0        im
1      None
2    luxury
3       hmv
4      None
Name: 5, dtype: object get       69
